Підключаємо модулі для роботи з даними

In [ ]:
!pip install urllib3 #might be useful

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import urllib3
from datetime import datetime
urllib3.disable_warnings()

Реалізуємо функцію, що буде завантажувати дані за номером області й називати файл відповідним чином, та функцію, що буде створювати датафрейм з відповідними даними.

In [ ]:
def load_data_VHI(region_ID): #downloads 'Area-Avegaed' and 'Percentage-Of-Area' data
  id_correction = [22, 24, 23, 25, 3, 4, 8, 19, 20, 21, 9, 26, 10, 11, 12, 13, 14, 15, 16, 27, 17, 18, 6, 1, 2, 7, 5]
  path = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_ID}&year1=1981&year2=2022&type=Mean"
  http = urllib3.PoolManager()
  req = http.request("GET", path, preload_content=False)
  region_ID = id_correction[region_ID-1]
  date = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
  filepath_VHI = f'vhi_id={region_ID}_{date}.csv' #'Area-Avegaed' data filepath
  out = open(filepath_VHI,'wb')
  out.write(req.read())
  out.close()
  path = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_ID}&year1=1981&year2=2022&type=VHI_Parea"
  req = http.request("GET", path, preload_content=False)  
  filepath_VHI_area = f'vhi_area_id={region_ID}_{date}.csv'
  out = open(filepath_VHI_area,'wb')
  out.write(req.read())
  out.close()
  return filepath_VHI, filepath_VHI_area

def VHI_dataframe(filepath, *, only_VHI=False):
  df = pd.read_csv(filepath,index_col=False, header=1) #'Area-Avegaed' dataframe
  df.rename(columns={' SMN': 'SMN', ' VHI<br>' : 'VHI'}, inplace=True) #hardcode
  if only_VHI: #if we interested only in VHI data with dates, of course
    return df.iloc[:,[0,1,6]]
  return df

def VHI_area_dataframe(filepath):
  name = ['year', 'week', '[0 5)', '[5 10)', '[10 15)', '[15 20)', '[20 25)', '[25 30)', '[30 35)', '[35 40)', '[40 45)', '[45 50)', '[50 55)', '[55 60)', '[60 65)', '[65 70)', '[70 75)', '[75 80)', '[80 85)', '[85 90)', '[90 95)','[95, 100)' , '[>=100]']
  df = pd.read_csv(filepath,index_col=False, on_bad_lines='skip', header = 2, names = name)
  df.iloc[0, 0] = '1982'
  df = df.iloc[:, :9] #'Percentage-Of-Area' dataframe (for droughts)
  return df

Завантажимо дані обраної області

In [ ]:
filepath_VHI, filepath_VHI_area = load_data_VHI(3)
print(filepath_VHI, filepath_VHI_area, sep='/n')
VHI_data = VHI_dataframe(filepath_VHI)
VHI_area_data = VHI_area_dataframe(filepath_VHI_area)

vhi_id=23_24-10-2022_07-07-52.csv/nvhi_area_id=23_24-10-2022_07-07-52.csv


Реалізуємо процедури для формування наступних вибірок:

*   Ряд VHI для області за рік, пошук екстремумів (min та max);
*   Ряд VHI за всі роки для області, виявити роки з екстремальними
  посухами, які торкнулися більше вказаного відсотка області;  
* Ряд VHI за всі роки для області, виявити роки з помірними
  посухами, які торкнулися більше вказаного відсотка області;

In [ ]:
def extr(data, year):
  index = int(year) - 1982
  new_data = data.iloc[index*52 : 52 + index*52]
  max, min = 0, 1000
  for value in new_data['VHI']:
    if value > max:
      max = value
  for value in new_data['VHI']:
    if value < min and value >= 0:
      min = value
  return min, max

def extr_drought(data, percentage):
  data = data.iloc[:, :5]
  year_list = []
  for date in range(len(data)):
    area = sum(data.iloc[date, 2:5])
    if area > percentage:
        year_list.append( ( int(data.iloc[date, 0]), int(data.iloc[date, 1]), f'area={area}'))
  return sorted(list(set(year_list)))

def modest_drought(data, percentage):
  year_list = []
  for date in range(len(data)):
    area = sum(data.iloc[date, 2:9])
    if area > percentage:
        year_list.append( ( int(data.iloc[date, 0]), int(data.iloc[date, 1]), f'area={area}'))
  return sorted(list(set(year_list)))

def get_year(dataframe, year, *, VHI=False):
  index = int(year) - 1982
  if VHI:
    return dataframe.iloc[52*index : 52 + 52*index, [0, 1, 6]]
  return dataframe.iloc[52*index : 52 + 52*index]

def get_value(dataframe, year, week, *, extr=False):
  if extr:
    dataframe = dataframe.iloc[:, 0:5]
  return dataframe[ (dataframe['year'] == f'{year}') & (dataframe['week'] == np.float64(week))]

Перевіримо коректнусть роботи функцій.
Можемо перевірити це, переглянувши відповідні датафрейми. Почнемо з перевірки коректності знаходження екстремумів за рік

In [ ]:
print(extr(VHI_data, 1984))
print(get_year(VHI_data, 1984, VHI=True))

(32.46, 60.82)
     year  week    VHI
104  1984   1.0  36.80
105  1984   2.0  40.06
106  1984   3.0  42.65
107  1984   4.0  42.73
108  1984   5.0  43.61
109  1984   6.0  44.38
110  1984   7.0  45.23
111  1984   8.0  46.32
112  1984   9.0  46.10
113  1984  10.0  45.68
114  1984  11.0  44.05
115  1984  12.0  40.87
116  1984  13.0  37.62
117  1984  14.0  36.26
118  1984  15.0  36.40
119  1984  16.0  35.38
120  1984  17.0  34.87
121  1984  18.0  37.96
122  1984  19.0  41.04
123  1984  20.0  44.63
124  1984  21.0  48.36
125  1984  22.0  51.36
126  1984  23.0  55.17
127  1984  24.0  58.55
128  1984  25.0  58.71
129  1984  26.0  57.59
130  1984  27.0  56.18
131  1984  28.0  57.31
132  1984  29.0  57.70
133  1984  30.0  58.11
134  1984  31.0  57.55
135  1984  32.0  58.87
136  1984  33.0  60.82
137  1984  34.0  60.43
138  1984  35.0  57.23
139  1984  36.0  51.89
140  1984  37.0  47.16
141  1984  38.0  43.04
142  1984  39.0  41.26
143  1984  40.0  44.09
144  1984  41.0  44.61
145  1984  42.0  42

Так, значення менmше за 32.46 і більше за 60.82 немає.
Тепер переглянемо один із років, щоб перевірити, чи дійсно в нас відповідну частину тереторії покривав певний тип посухи.



In [ ]:
print(modest_drought(VHI_area_data, 40)[0])#remove '[0]' to see full list
print(get_value(VHI_area_data, 1982, 5))

(1982, 5, 'area=54.28')
   year  week  [0 5)  [5 10)  [10 15)  [15 20)  [20 25)  [25 30)  [30 35)
4  1982   5.0    0.0    0.09     0.56     3.35    10.24    17.32    22.72


Так, дійсно, 1982 року 5-ї неделі, наш регіон був більше ніж на 40% покритий помірними посухами.
Тепер переіримо для екстремальної посухи.

In [ ]:
print(extr_drought(VHI_area_data, 50)) #remove '[0]' to see full list
print( get_value(VHI_area_data, 1984, 49))

[(1984, 49, 'area=51.86'), (1993, 51, 'area=51.77'), (1993, 52, 'area=51.12'), (2000, 45, 'area=53.17'), (2000, 46, 'area=55.58'), (2000, 47, 'area=53.81999999999999')]
     year  week  [0 5)  [5 10)  [10 15)  [15 20)  [20 25)  [25 30)  [30 35)
152  1984  49.0  11.45   18.62    21.79    21.14     13.5      5.4     3.07


Так, функція нас не обманює: дійсно, 1984 року 46-ї неділі, наш регіон покривала екстремальна посуха більш ніж на 30%.